Sequence to Sequence Translation.

##**Character level Model**



Using Encoder and Decoder model of LSTM.
Encoder produces the internal state which is context or conditioning.
Decoder predict the next character of target sequence given the previous character.

Teacher Forcing Method

Importantly, the encoder uses as initial state the state vectors from the encoder, which is how the decoder obtains information about what it is supposed to generate. Effectively, the decoder learns to generate targets[t+1...] given targets[...t], conditioned on the input sequence.

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 20000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'deu.txt'

In [ ]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')  
for line in lines[: min(num_samples, len(lines) - 1)]:
  input_text, target_text, _ = line.split('\t')
  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [ ]:
print(input_texts)
print(target_texts)


['Go.', 'Hi.', 'Hi.', 'Run!', 'Run.', 'Wow!', 'Wow!', 'Fire!', 'Help!', 'Help!', 'Stop!', 'Stop!', 'Wait!', 'Wait.', 'Begin.', 'Go on.', 'Hello!', 'Hello!', 'Hurry!', 'Hurry!', 'I hid.', 'I hid.', 'I ran.', 'I see.', 'I see.', 'I try.', 'I won!', 'I won!', 'Relax.', 'Shoot!', 'Shoot!', 'Smile.', 'Ask me.', 'Ask me.', 'Ask me.', 'Attack!', 'Attack!', 'Cheers!', 'Eat it.', 'Eat up.', 'Eat up.', 'Freeze!', 'Freeze!', 'Go now.', 'Got it!', 'Got it!', 'Got it!', 'Got it?', 'Got it?', 'Got it?', 'He ran.', 'He ran.', 'Hop in.', 'Hop in.', 'Hug me.', 'Hug me.', 'Hug me.', 'I care.', 'I fell.', 'I fell.', 'I fell.', 'I fell.', 'I fell.', 'I fled.', 'I fled.', 'I know.', 'I lied.', 'I lost.', 'I paid.', 'I paid.', 'I sang.', 'I spit.', 'I spit.', 'I swim.', 'I wept.', 'I wept.', "I'm 19.", "I'm 19.", "I'm OK.", "I'm OK.", "I'm up.", "I'm up.", 'Listen.', 'No way!', 'No way!', 'No way!', 'No way!', 'No way!', 'Really?', 'Really?', 'Really?', 'Thanks.', 'Try it.', 'We try.', 'We won.', 'Why me?',

In [ ]:
print(input_characters)
print(target_characters)
print(len(input_characters))
print(len(target_characters))

{'.', 'h', '7', 'j', 'J', ':', 'U', 'Q', 'A', 'l', 'd', 'E', 'f', 'q', 'o', 'L', 'z', 'y', 'x', '3', '8', 'b', 'P', ' ', 'H', 's', '0', 'w', 'C', 'V', 'm', 'M', 'a', 'k', 'i', 'I', 'Y', 'u', '$', '5', '!', '9', 'R', 'n', 'N', 'T', '4', 'c', "'", 'g', 'S', 'O', 'r', 'W', 'G', 't', '?', '1', ',', '2', 'B', '"', 'v', 'e', 'F', '6', 'p', '%', '-', 'K', 'D'}
{'h', 'ö', 'd', 'f', 'q', 'o', 'L', 'w', 'M', 'Z', 'N', 'Ö', 'O', '\n', '1', 'ü', '"', 'F', 'p', 'Q', 'é', 'E', 'Ü', 'P', 'H', 's', '0', 'C', 'V', 'ß', 'i', '$', '!', '9', '–', '’', '4', 'c', "'", 'r', 'G', '\xa0', 'e', 'K', '7', '\t', 'z', 'y', 'b', '5', '\u202f', 'ä', 'n', 'T', '2', 'S', '?', 'ō', ',', '“', '6', '%', '-', 'j', '.', 'J', ':', 'U', 'A', 'l', '„', 'x', '3', '8', 'Ä', ' ', 'm', 'a', 'k', 'I', 'Y', 'u', 'R', 'g', 'W', 't', 'B', 'v', 'D'}
71
89


In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [ ]:
print('Number of samples: ', len(input_texts))
print('Number of unique input token: ', num_encoder_tokens)
print('Number of unique output token: ', num_decoder_tokens)
print('Max Seq length for inputs: ', max_encoder_seq_length)
print('Max seq length for outputs: ', max_decoder_seq_length)

Number of samples:  20000
Number of unique input token:  71
Number of unique output token:  89
Max Seq length for inputs:  17
Max seq length for outputs:  74


In [ ]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])


In [ ]:
print(input_token_index)
print(target_token_index)

{' ': 0, '!': 1, '"': 2, '$': 3, '%': 4, "'": 5, ',': 6, '-': 7, '.': 8, '0': 9, '1': 10, '2': 11, '3': 12, '4': 13, '5': 14, '6': 15, '7': 16, '8': 17, '9': 18, ':': 19, '?': 20, 'A': 21, 'B': 22, 'C': 23, 'D': 24, 'E': 25, 'F': 26, 'G': 27, 'H': 28, 'I': 29, 'J': 30, 'K': 31, 'L': 32, 'M': 33, 'N': 34, 'O': 35, 'P': 36, 'Q': 37, 'R': 38, 'S': 39, 'T': 40, 'U': 41, 'V': 42, 'W': 43, 'Y': 44, 'a': 45, 'b': 46, 'c': 47, 'd': 48, 'e': 49, 'f': 50, 'g': 51, 'h': 52, 'i': 53, 'j': 54, 'k': 55, 'l': 56, 'm': 57, 'n': 58, 'o': 59, 'p': 60, 'q': 61, 'r': 62, 's': 63, 't': 64, 'u': 65, 'v': 66, 'w': 67, 'x': 68, 'y': 69, 'z': 70}
{'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, '$': 5, '%': 6, "'": 7, ',': 8, '-': 9, '.': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, '?': 22, 'A': 23, 'B': 24, 'C': 25, 'D': 26, 'E': 27, 'F': 28, 'G': 29, 'H': 30, 'I': 31, 'J': 32, 'K': 33, 'L': 34, 'M': 35, 'N': 36, 'O': 37, 'P': 38, 'Q': 39, 'R': 40, 'S': 41,

In [ ]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

Converting Each sentences into one hot encoding
of size of number of unique characters in particular language and lenght of each sentence

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [ ]:
encoder_input_data.shape

(20000, 17, 71)

In [ ]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
250/250 [==============================] - 6s 17ms/step - loss: 0.4613 - accuracy: 0.1574 - val_loss: 0.5253 - val_accuracy: 0.1777
Epoch 2/100
250/250 [==============================] - 3s 14ms/step - loss: 0.4435 - accuracy: 0.1605 - val_loss: 0.5309 - val_accuracy: 0.1767
Epoch 3/100
250/250 [==============================] - 3s 14ms/step - loss: 0.4325 - accuracy: 0.1624 - val_loss: 0.5179 - val_accuracy: 0.1791
Epoch 4/100
250/250 [==============================] - 3s 14ms/step - loss: 0.4275 - accuracy: 0.1641 - val_loss: 0.5064 - val_accuracy: 0.1821
Epoch 5/100
250/250 [==============================] - 4s 14ms/step - loss: 0.4193 - accuracy: 0.1660 - val_loss: 0.4948 - val_accuracy: 0.1841
Epoch 6/100
250/250 [==============================] - 3s 14ms/step - loss: 0.4128 - accuracy: 0.1676 - val_loss: 0.5007 - val_accuracy: 0.1826
Epoch 7/100
250/250 [==============================] - 3s 14ms/step - loss: 0.4067 - accuracy: 0.1685 - val_loss: 0.4851 - val_accuracy:

In [ ]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Gen.

-
Input sentence: Hi.
Decoded sentence: Halle.

-
Input sentence: Hi.
Decoded sentence: Halle.

-
Input sentence: Run!
Decoded sentence: Faufen.

-
Input sentence: Run.
Decoded sentence: Laufen.

-
Input sentence: Wow!
Decoded sentence: Wann in der das.

-
Input sentence: Wow!
Decoded sentence: Wann in der das.

-
Input sentence: Fire!
Decoded sentence: Feler.

-
Input sentence: Help!
Decoded sentence: Hull.

-
Input sentence: Help!
Decoded sentence: Hull.

-
Input sentence: Stop!
Decoded sentence: Schrier.

-
Input sentence: Stop!
Decoded sentence: Schrier.

-
Input sentence: Wait!
Decoded sentence: Warte.

-
Input sentence: Wait.
Decoded sentence: Warten.

-
Input sentence: Begin.
Decoded sentence: Nimm das aus.

-
Input sentence: Go on.
Decoded sentence: Geh nicht.

-
Input sentence: Hello!
Decoded sentence: Serde.

-
Input sentence: Hello!
Decoded sentence: Serde.

-
Input sentence: Hurry!
Decoded sentence: Schner.

-
Input sentence: Hu